In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib as plt

In [ ]:
df  = pd.read_csv('../input/star-type-classification/Stars.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape 

In [ ]:
print(df['Color'].unique())
print(df['Spectral_Class'].unique())

In [ ]:
df.isnull().sum()

In [ ]:
df.Color.value_counts().sort_values(ascending=False)

In [ ]:
col = [x for x in df.Color.value_counts().sort_values(ascending=False).head(17).index]
col

In [ ]:
for label in col:
    df[label] = np.where(df['Color']==label , 1, 0)
df = df.drop(['Color'], axis=1)

In [ ]:
df.head(5)

In [ ]:
df = df.drop(['Blue White',
'Blue-White',
'Blue white',
'white',
'White-Yellow',
'Yellowish',], axis=1)
df = df.rename(columns = {'Pale yellow orange':'Pale_yellow_orange','Yellowish White':'Yellowish_White','yellow-white':'yellow_white','Blue-white':'Blue_white'}, inplace = False)

In [ ]:
df.Spectral_Class.value_counts().sort_values(ascending=False)

In [ ]:
cal = [x for x in df.Spectral_Class.value_counts().sort_values(ascending=False).head(7).index]
cal

In [ ]:
for label in cal:
    df[label] = np.where(df['Spectral_Class']==label , 1, 0)
df = df.drop(['Spectral_Class'], axis=1)

In [ ]:
df.head(5)

In [ ]:
x = df.iloc[:,0:]
y = df.iloc[:,4]

In [ ]:
x = x.drop(['Type'], axis=1)

In [ ]:
x.head(3)

In [ ]:
y.head(3)

In [ ]:
#feature Importance 
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
feat = pd.Series(model.feature_importances_ , index = x.columns)
feat.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x , y , test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_ran = RandomForestRegressor()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Randomized Search CV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
predictions=rf_random.predict(x_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))